<a href="https://colab.research.google.com/github/MaJu502/proyectoDL/blob/main/Resnet_pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Proyecto Deep Learning
- Diego Cordova 20212
- Marco Jurado 20308
- Cristian Aguirre 20231
- Paola Contreras 20213
- Paola de Leon 20361

## Carga de Dataset

In [1]:
!gdown 1BoNWFSXxtN74PgLcBxfGWyeKUOfevQEP

Downloading...
From: https://drive.google.com/uc?id=1BoNWFSXxtN74PgLcBxfGWyeKUOfevQEP
To: /content/kaggle.json
100% 64.0/64.0 [00:00<00:00, 234kB/s]


In [2]:
! pip install kaggle

In [3]:
!ls -a /root/

.   .bashrc  .config   .jupyter  .launchpadlib	.npm	  .tmux.conf
..  .cache   .ipython  .keras	 .local		.profile  .wget-hsts


In [4]:
!mkdir /root/.kaggle/

In [5]:
!mv /content/kaggle.json /root/.kaggle/


In [6]:
ls /root/.kaggle/


kaggle.json


In [7]:
!chmod 600 /root/.kaggle/kaggle.json

In [8]:
!kaggle datasets download -d jonathanoheix/face-expression-recognition-dataset

 92% 111M/121M [00:01<00:00, 127MB/s] 
100% 121M/121M [00:01<00:00, 112MB/s]


In [9]:
%cd /content
!unzip -q face-expression-recognition-dataset.zip

/content


# Modelo

In [10]:
import os
import shutil

# Ruta de la carpeta que deseas eliminar
folder_path = './images/train/disgust'

# Elimina todas las imágenes en la carpeta
for file_name in os.listdir(folder_path):
    file_path = os.path.join(folder_path, file_name)
    try:
        if os.path.isfile(file_path) or os.path.islink(file_path):
            os.unlink(file_path)
        elif os.path.isdir(file_path):
            shutil.rmtree(file_path)
    except Exception as e:
        print(f'Error al eliminar {file_path}: {e}')

# Elimina la carpeta misma
try:
    os.rmdir(folder_path)
except Exception as e:
    print(f'Error al eliminar la carpeta {folder_path}: {e}')


In [11]:
# Ruta de la carpeta que deseas eliminar
folder_path = './images/validation/disgust'

# Elimina todas las imágenes en la carpeta
for file_name in os.listdir(folder_path):
    file_path = os.path.join(folder_path, file_name)
    try:
        if os.path.isfile(file_path) or os.path.islink(file_path):
            os.unlink(file_path)
        elif os.path.isdir(file_path):
            shutil.rmtree(file_path)
    except Exception as e:
        print(f'Error al eliminar {file_path}: {e}')

# Elimina la carpeta misma
try:
    os.rmdir(folder_path)
except Exception as e:
    print(f'Error al eliminar la carpeta {folder_path}: {e}')

In [43]:
import os
import random
import shutil

# Ruta de la carpeta que deseas modificar
folder_path = './images/train/happy'

# Lista de archivos en la carpeta
file_list = os.listdir(folder_path)

# Número de imágenes que deseas eliminar
num_images_to_delete = 3000

# Verifica si hay suficientes imágenes para eliminar
if len(file_list) < num_images_to_delete:
    print(f'No hay suficientes imágenes en {folder_path} para eliminar.')
else:
    # Selecciona aleatoriamente las imágenes que se eliminarán
    images_to_delete = random.sample(file_list, num_images_to_delete)

    # Elimina las imágenes seleccionadas
    for file_name in images_to_delete:
        file_path = os.path.join(folder_path, file_name)
        try:
            if os.path.isfile(file_path):
                os.unlink(file_path)
            elif os.path.isdir(file_path):
                shutil.rmtree(file_path)
        except Exception as e:
            print(f'Error al eliminar {file_path}: {e}')

    print(f'Se han eliminado {num_images_to_delete} imágenes de {folder_path}.')


Se han eliminado 3000 imágenes de ./images/train/happy.


In [13]:
import cv2
import os
import numpy as np

# Directorio raíz que contiene subdirectorios para cada emoción
root_dir = './images/train/'

# Lista de emociones
emotions = ['angry', 'fear', 'happy', 'neutral', 'sad', 'surprise']

# Inicializa un diccionario para almacenar shapes únicos por emoción
unique_shapes_by_emotion = {emotion: [] for emotion in emotions}

# Itera a través de las carpetas de emociones
for emotion in emotions:
    emotion_dir = os.path.join(root_dir, emotion)
    image_files = os.listdir(emotion_dir)

    # Itera a través de los archivos de imágenes en cada carpeta de emoción
    for image_file in image_files:
        image_path = os.path.join(emotion_dir, image_file)
        image = cv2.imread(image_path)
        shape = image.shape
        if shape not in unique_shapes_by_emotion[emotion]:
            unique_shapes_by_emotion[emotion].append(shape)

# Imprime los shapes únicos por emoción
for emotion, shapes in unique_shapes_by_emotion.items():
    print(f"Shapes únicos para la emoción '{emotion}':")
    for shape in shapes:
        print(shape)


Shapes únicos para la emoción 'angry':
(48, 48, 3)
Shapes únicos para la emoción 'fear':
(48, 48, 3)
Shapes únicos para la emoción 'happy':
(48, 48, 3)
Shapes únicos para la emoción 'neutral':
(48, 48, 3)
Shapes únicos para la emoción 'sad':
(48, 48, 3)
Shapes únicos para la emoción 'surprise':
(48, 48, 3)


## Loader para el Modelo

In [53]:
import torch
from torchvision import datasets, transforms
from torchvision.datasets import ImageFolder
from torchvision import transforms as T

batch_size = 128
base_path = "./images/"

train_augs = T.Compose([T.RandomHorizontalFlip(p = 0.5),T.RandomRotation(degrees = (-20,+20)),T.ToTensor()])
#To tensor in pytorch converts image from numpy or PIL into pytorch tensors also it will convert (h,w,c)->(c,h,w)
valid_augs = T.Compose([T.ToTensor()])


# Crear conjuntos de datos
train_dataset = ImageFolder(base_path + "train", transform =train_augs)
validation_dataset = ImageFolder(base_path + "validation", transform =valid_augs)

# Crear generadores de lotes (DataLoaders)
trainloader = torch.utils.data.DataLoader(train_dataset,batch_size=batch_size,shuffle = True)
validloader = torch.utils.data.DataLoader(validation_dataset,batch_size=batch_size)

## Se preparan los datos para CUDA

## Entreno del Modelo

In [ ]:
!pip install scikit-learn

In [ ]:
!pip install timm

In [54]:
import timm
from torch import nn

class FaceModel(nn.Module):
    def __init__(self):
        super(FaceModel,self).__init__()
        self.eff_net = timm.create_model('resnet34',pretrained = True,num_classes = 7)
    def forward(self,images,labels = None):
        logits = self.eff_net(images)
        if labels != None:
            loss = nn.CrossEntropyLoss()(logits,labels)
            return logits,loss
        return logits

modelo = FaceModel()

In [55]:
device = ('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Computation device: {device}\n")
modelo = modelo.to(device)

device_in_model = 'cpu' if False in [i.is_cuda for i in modelo.parameters()] else 'cuda'
print('model using', device_in_model)

Computation device: cuda

model using cuda


In [65]:
def multiclass_accuracy(y_pred,y_true):
    top_p,top_class = y_pred.topk(1,dim = 1)
    equals = top_class == y_true.view(*top_class.shape)
    return torch.mean(equals.type(torch.FloatTensor))

In [66]:
from tqdm import tqdm

In [67]:
epoch = 20


def train_fn(model,dataloader,optimizer,current_epo):
    model.train()
    total_loss = 0.0
    total_acc = 0.0
    tk = tqdm(dataloader,desc = "EPOCHS" + "[TRAIN]" + str(current_epo+1) + "/"+ str(epoch))
    for t,data in enumerate(tk):
        images,labels = data
        images,labels = images.to(device),labels.to(device)

        optimizer.zero_grad()
        logits,loss = model(images,labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
        total_acc += multiclass_accuracy(logits,labels)
        tk.set_postfix({'loss':'%6f' %float(total_loss/(t+1)),'acc':'%6f' %float(total_acc/(t+1))})
    return total_loss/len(dataloader),total_acc/len(dataloader)



In [68]:


def eval_fn(model,dataloader,current_epo):
    model.eval()
    total_loss = 0.0
    total_acc = 0.0
    tk = tqdm(dataloader,desc = "EPOCH" + "[VALID]" + str(current_epo + 1) + "/"+ str(epoch))
    for t,data in enumerate(tk):
        images,labels = data
        images,labels = images.to(device),labels.to(device)


        logits,loss = model(images,labels)

        total_loss += loss.item()
        total_acc += multiclass_accuracy(logits,labels)
        tk.set_postfix({'loss':'%6f' %float(total_loss/(t+1)),'acc':'%6f' %float(total_acc/(t+1))})
    return total_loss/len(dataloader),total_acc/len(dataloader)



In [69]:
optimizer = torch.optim.Adam(modelo.parameters(),lr = 0.0001)

In [70]:
best_valid_loss= np.inf
for i in range(epoch):
    train_loss,train_acc = train_fn(modelo,trainloader,optimizer,i)
    eval_loss,eval_acc = eval_fn(modelo,validloader,i)
    if eval_loss < best_valid_loss:
        torch.save(modelo.state_dict(),'best-weights.pt')
        print("Saved Best Valid Loss")
        best_valid_loss = eval_loss

EPOCH[VALID]1/20: 100%|██████████| 55/55 [00:03<00:00, 16.21it/s, loss=1.748013, acc=0.297648]


Saved Best Valid Loss


EPOCH[VALID]2/20: 100%|██████████| 55/55 [00:03<00:00, 15.60it/s, loss=1.687636, acc=0.330589]


Saved Best Valid Loss


EPOCH[VALID]3/20: 100%|██████████| 55/55 [00:03<00:00, 16.04it/s, loss=1.632620, acc=0.354311]


...

EPOCH[VALID]16/20: 100%|██████████| 55/55 [00:03<00:00, 14.92it/s, loss=1.257842, acc=0.533718]


Saved Best Valid Loss


EPOCH[VALID]19/20: 100%|██████████| 55/55 [00:04<00:00, 13.14it/s, loss=1.251940, acc=0.542809]


Saved Best Valid Loss


EPOCH[VALID]20/20: 100%|██████████| 55/55 [00:03<00:00, 13.81it/s, loss=1.264448, acc=0.541091]


In [50]:
import pickle

model_filename = "model.pkl"

with open(model_filename, 'wb') as model_file:
    pickle.dump(modelo, model_file)


In [51]:
import matplotlib.pyplot as plt

def plot_results(history):
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
    loss = history.history['loss']
    val_loss = history.history['val_loss']

    epochs = range(1, len(acc) + 1)

    plt.figure(figsize = (24, 6))
    plt.subplot(1,2,1)
    plt.plot(epochs, acc, 'b', label = 'Training Accuracy')
    plt.plot(epochs, val_acc, 'r', label = 'Validation Accuracy')
    plt.grid(True)
    plt.legend()
    plt.xlabel('Epoch')



    plt.subplot(1,2,2)
    plt.plot(epochs, loss, 'b', label = 'Training Loss')
    plt.plot(epochs, val_loss, 'r', label = 'Validation Loss')
    plt.grid(True)
    plt.legend()
    plt.xlabel('Epoch')
    plt.show()

def get_best_epcoh(history):
    valid_acc = history.history['val_accuracy']
    best_epoch = valid_acc.index(max(valid_acc)) + 1
    best_acc =  max(valid_acc)
    print('Best Validation Accuracy Score {:0.5f}, is for epoch {}'.format( best_acc, best_epoch))
    return best_epoch